In [63]:
import numpy as np
from sklearn.manifold import TSNE

 
import pandas as pd
country = pd.read_csv('Data/Countrys.csv')
country.head()

,Unnamed: 0,content,sentiment
0,0,Yo espero tranquila. Si gana de nvo alg partid...,NEU
1,1,"Los fracasos de Michelle, Cristina y Dilma mue...",N
2,2,Debemos orar por los hnos venezolanos gente q ...,N
3,3,Me dijo que no le había gustado no se cuantas ...,NEU
4,4,@susanagkp jajaja yo fui el viernes ! precavid...,N


In [64]:
countryContSen = country[['content','sentiment']].copy()
countryContSen['sentiment'] = countryContSen.sentiment.map({'NONE':0,'N':1,'NEU':2, 'P':3})
countryContSen

,content,sentiment
0,Yo espero tranquila. Si gana de nvo alg partid...,2
1,"Los fracasos de Michelle, Cristina y Dilma mue...",1
2,Debemos orar por los hnos venezolanos gente q ...,1
3,Me dijo que no le había gustado no se cuantas ...,2
4,@susanagkp jajaja yo fui el viernes ! precavid...,1
...,...,...
3670,Quiero como tener carro para hacer todo rápido,2
3671,Tristeza más hp me da siempre en año nuevo,1
3672,no encuentro ninguna buena serie en NF.,1
3673,@Alphasniper97 Alpha suerte y ten un feliz año...,3


In [69]:
# Almacenamiento de stop_words
import spacy
documents = countryContSen['content'].to_list()
nlp = spacy.load("es_core_news_sm")
stopWordsSpanish = []

for text in documents:
    myDoc = nlp(text)
    for word in myDoc:
        if word.is_stop is True and word.text not in stopWordsSpanish:
            stopWordsSpanish.append(word.text)
            #print(word.text)

        if word.like_num is True and word.text not in stopWordsSpanish:
            stopWordsSpanish.append(word.text)
            #print(word.text)

        if word.is_punct is True and word.text not in stopWordsSpanish:
            stopWordsSpanish.append(word.text)
            #print(word.text)

        if any(chr.isdigit() or chr == '_' for chr in word.text) and word.text not in stopWordsSpanish:
            stopWordsSpanish.append(word.text)
            #print(word.text)

stopWordsSpanish

['Yo',
 '.',
 'Si',
 'de',
 ',',
 'después',
 'sé',
 'mal',
 'pero',
 'me',
 'voy',
 'a',
 'bastante',
 'Los',
 'y',
 'que',
 'no',
 'sólo',
 'las',
 'son',
 'sino',
 'tan',
 'bien',
 'por',
 'los',
 'es',
 'hoy',
 'la',
 'en',
 'su',
 'tengo',
 '...',
 'Me',
 'dijo',
 'le',
 'había',
 'se',
 'cuantas',
 'estuvo',
 'todo',
 'el',
 'eso',
 'yo',
 'fui',
 '!',
 'pasado',
 'ir',
 'ya',
 'esta',
 'así',
 'otro',
 'ese',
 'mi',
 'esa',
 'fue',
 'un',
 'muy',
 'bueno',
 'desde',
 '21',
 '28',
 'al',
 'porque',
 'unas',
 'igual',
 'si',
 'pueden',
 'sola',
 'está',
 'Está',
 'demás',
 'mucha',
 'soy',
 'Saben',
 'donde',
 'sus',
 '?',
 'ahí',
 'les',
 '@letsgroove27',
 'estaba',
 'era',
 'fuera',
 '....',
 'o',
 '-',
 'Para',
 'mí',
 'Entre',
 'estoy',
 'La',
 'con',
 'todos',
 'mis',
 'Otro',
 'vez',
 'mejor',
 'No',
 'otra',
 '@veronica_alonso',
 'dice',
 'existe',
 'para',
 'tampoco',
 'día',
 '"',
 'tu',
 'tenemos',
 'del',
 'ni',
 'Hoy',
 'YA',
 'ME',
 'DE',
 'qué',
 '@Adri_tranqui',
 'm

In [71]:

# Inicialización del Contador de Vectorización
count_vector = CountVectorizer(stop_words=stopWordsSpanish)

In [72]:
# Lista de palabrás únicas
count_vector.fit(documents)
names = count_vector.get_feature_names()
names

/home/nickgrite51/Entornos/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['00', '000', '00am', '02', '03', '04', '0hmyvolunteer', '102', '10_nov', '10s', '143', '150', '1559', '1d', '1maaariii1', '204danielle', '20s', '22', '2225', '2820gchu', '2891891cap', '2c', '2tes', '3080', '30am', '30hs', '30pm', '3k', '46pm', '55', '5honamas', '5nfidente', '60', '7000', '74evmm', '80', '87', '8827', '90feliznavidad', '99allison11', '9km', '__el_negro', '__josejuan', '__walter', '_adrianaquevedo', '_alejandrodp', '_alineepa', '_anatabeira', '_cmere', '_fer_alonso', '_fersha', '_josejones', '_mariajosee', '_melibarra', '_monsejimenezp', '_nosoyartista', '_pepepar', '_star_vader', '_suniac', '_void07', 'abbu25', 'abby_maff', 'abiel_vasquez', 'absa_lon', 'acompañarnos', 'adri_hdezv', 'adri_tranqui', 'adriantr09', 'afigue2010', 'agronegocios_uy', 'agusblanco01', 

['00',
 '000',
 '00am',
 '02',
 '03',
 '04',
 '0hmyvolunteer',
 '102',
 '10_nov',
 '10am',
 '10s',
 '11pm',
 '143',
 '150',
 '1559',
 '1cm',
 '1d',
 '1k',
 '1maaariii1',
 '204danielle',
 '20s',
 '22',
 '2225',
 '2820gchu',
 '2891891cap',
 '2c',
 '2pm',
 '2tes',
 '3080',
 '30am',
 '30cm',
 '30hs',
 '30pm',
 '3k',
 '3pm',
 '46pm',
 '4am',
 '4g',
 '4k',
 '55',
 '5honamas',
 '5nfidente',
 '60',
 '6pm',
 '7000',
 '74evmm',
 '7am',
 '80',
 '87',
 '8827',
 '8am',
 '8g',
 '90feliznavidad',
 '99allison11',
 '9am',
 '9km',
 '__',
 '__el_negro',
 '__josejuan__',
 '__walter__',
 '_adrianaquevedo',
 '_alejandrodp',
 '_alineepa',
 '_anatabeira',
 '_cmere',
 '_fer_alonso',
 '_fersha_',
 '_josejones',
 '_mariajosee_',
 '_melibarra',
 '_monsejimenezp',
 '_nosoyartista',
 '_pepepar',
 '_star_vader',
 '_suniac',
 '_tener',
 '_void07',
 'aaa',
 'aaaa',
 'aaaaaaah',
 'aaah',
 'aaahhhhggg',
 'ab',
 'abajo',
 'abandonada',
 'abandonado',
 'abarcara',
 'abbu25',
 'abby_maff',
 'abcdesignated',
 'abdiel',
 'ab

In [53]:
count_vector.fit(documents)
names = count_vector.get_feature_names()
names

/home/nickgrite51/Entornos/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '00am',
 '02',
 '03',
 '04',
 '095327750',
 '0hmyvolunteer',
 '10',
 '100',
 '1000',
 '102',
 '109',
 '10_nov',
 '10am',
 '10pax',
 '10s',
 '11',
 '111',
 '115',
 '11pm',
 '12',
 '12md',
 '13',
 '14',
 '143',
 '15',
 '150',
 '150kmph',
 '1559',
 '16',
 '168',
 '17',
 '18',
 '1965',
 '1cm',
 '1d',
 '1er',
 '1era',
 '1k',
 '1maaariii1',
 '1ra',
 '1ro',
 '20',
 '200',
 '2000',
 '2005',
 '2006',
 '2010',
 '2011',
 '2013',
 '2016',
 '2017',
 '2017prometoque',
 '2018',
 '204danielle',
 '20pax',
 '20s',
 '21',
 '22',
 '2225',
 '23',
 '24',
 '24hrs',
 '25',
 '26',
 '27',
 '28',
 '280',
 '2820gchu',
 '2891891cap',
 '29',
 '2c',
 '2do',
 '2hrs35min',
 '2pm',
 '2tes',
 '2x1',
 '30',
 '300',
 '3080',
 '30am',
 '30cm',
 '30hs',
 '30pm',
 '31',
 '32',
 '320',
 '330mil',
 '34',
 '35',
 '36',
 '39',
 '3d',
 '3gob',
 '3k',
 '3pm',
 '40',
 '400',
 '41',
 '4400',
 '45',
 '45min',
 '46pm',
 '49',
 '4am',
 '4g',
 '4k',
 '4to',
 '50',
 '500',
 '50mil',
 '55',
 '5comentarios',
 '5h',
 '5honam

In [73]:
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [74]:
frequency_matrix = pd.DataFrame(data=doc_array, columns=names)
frequency_matrix

,00,000,00am,02,03,04,0hmyvolunteer,102,10_nov,10am,...,úmica,única,únicamente,únicas,único,únicos,úrgeme,úselo,útil,útiles
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
frequency_matrix = pd.DataFrame(data=doc_array, columns=names)
frequency_matrix

,00,000,00am,02,03,04,095327750,0hmyvolunteer,10,100,...,úmica,única,únicamente,únicas,único,únicos,úrgeme,úselo,útil,útiles
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# Preparación de los datos
X_train, X_test, y_train, y_test = train_test_split(countryContSen['content'], countryContSen['sentiment'], random_state=1)

In [79]:

# Instanciar el metodo CountVector
count_vector = CountVectorizer()

# Ajuste los datos de entrenamiento y luego devuelva la matriz
training_data = count_vector.fit_transform(X_train)

# Transforme los datos de prueba y devuelva la matriz. Tener en cuenta que no se está ajustando los datos de prueba en CountVectorizer()
testing_data = count_vector.transform(X_test)

In [80]:
# Uso de k-Nearest Neighbor
knn = KNeighborsClassifier()
knn.fit(training_data, y_train)

KNeighborsClassifier()

In [90]:
# Predicción del modelo
predictions = knn.predict(testing_data)

In [93]:
# Exportación de los Modelos
import pickle
pickle.dump(count_vector, open('Modelos/vectorizerKNN.sav', 'wb'))
pickle.dump(knn, open('Modelos/classifierKNN.sav', 'wb'))

In [94]:
vectorizerKNN = pickle.load(open('Modelos/vectorizerKNN.sav', 'rb'))
classifierKNN = pickle.load(open('Modelos/classifierKNN.sav', 'rb'))

In [95]:
inp2 = "Feliz"
text_vectorKNN = vectorizerKNN.transform([inp2])
resultKNN = classifierKNN.predict(text_vectorKNN)

In [96]:
print(resultKNN[0])

3


In [ ]:
print(resultNB[0])